In [1]:
%%capture
!pip install jsonlines
!pip install evaluate
!pip install rouge_score
!pip install bert_score
!pip install unsloth
!pip install python-dotenv
!pip install neo4j
!pip install langchain
!pip install langchain_together
!unzip /content/results.zip

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`
Unsloth 2024.10.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [84]:
import langchain
from langchain_together import ChatTogether
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import os
from dotenv import load_dotenv
load_dotenv()

# os.environ["TOGETHER_API_KEY"] = os.getenv("TOGETHER_API_KEY")
os.environ["TOGETHER_API_KEY"] = "80f6eb25e6e817651c95012f6f92a9a4cfc33a7352e996ee46ca65fa4d7ce051"

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
              "Compare the following two lists. They are similar if they are almost identical, "
              "or if they convey the same information."
              "Respond with valid JSON {{\"similar\" : bool}}"
        ),
        (
            "human",
              "========== List 1 =========\n{list1}\n\n"
              "========== List 2 =========\n{list2}\n\n"
              "Can you tell me if these lists are similar?")
    ]
)

llm = ChatTogether(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    temperature=0.0,
    max_tokens=None,
    timeout=None,
    max_retries=5,
    # other params...
)

def same_meaning_metrics(list1, list2):
    chain = prompt | llm | JsonOutputParser()
    result = chain.invoke({"list1": str(list1).lower(), "list2": str(list2).lower()})
    return result

print(same_meaning_metrics(["Yasuo", "Yone"], ["Yone", "Yasuo"]))

{'similar': True}


In [79]:
import asyncio
import os
import uuid
from dotenv import load_dotenv

# NEO4J_URI = os.getenv("NEO4J_URI")
# NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
# NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
# NEO4J_DATABASE = os.getenv("NEO4J_DATABASE")
NEO4J_URI = "neo4j+s://e8dc6a79.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "IuDkOcNCHe_7HtondReVIStRGAyiUgzP8HVXnjU3h4w"
NEO4J_DATABASE = "neo4j"

from neo4j import GraphDatabase
import jsonlines
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

def connect_to_neo4j(uri, database, user, password):
    driver = GraphDatabase.driver(uri, database=database, auth=(user, password))
    return driver

def validate_query(driver, query):
    with driver.session() as session:
        try:
            # Query to fetch all Workplace nodes
            result = session.run(query)
            return True, list(result)
        except Exception as e:
            # print(f"An error occurred: {e}")
            return False, []

def success_execute_metrics(predictions, references):
    valid_set = []
    invalid_set = []
    same_meaning_set = []
    different_meaning_set = []

    driver = connect_to_neo4j(
        NEO4J_URI, NEO4J_DATABASE, NEO4J_USERNAME, NEO4J_PASSWORD
    )

    for prediction, reference in zip(predictions, references):

        valid, prediction_result = validate_query(driver, prediction)

        if valid:
            valid_set.append(prediction)
            _, reference_result = validate_query(driver, reference)

            if same_meaning_metrics(prediction_result, reference_result)["similar"]:
                same_meaning_set.append(prediction)
            else:
                different_meaning_set.append(prediction)
        else:
            invalid_set.append(prediction)


    driver.close()  # It's good practice to close the driver when done

    percent_valid = len(valid_set)/len(valid_set + invalid_set)
    percent_invalid = len(invalid_set)/len(valid_set + invalid_set)
    percent_same_meaning = len(same_meaning_set)/len(same_meaning_set + different_meaning_set)
    percent_different_meaning = len(different_meaning_set)/len(same_meaning_set + different_meaning_set)

    return {
        "percent_valid": percent_valid,
        "percent_invalid": percent_invalid,
        "percent_same_meaning": percent_same_meaning,
        "percent_different_meaning": percent_different_meaning,
        }

In [80]:
import evaluate
import numpy as np

# Load metrics
exact_match_metric = evaluate.load("exact_match")
bertscore_metric = evaluate.load("bertscore")

def compute_metrics(predictions, references):
    success_execute = success_execute_metrics(predictions, references)
    bertscore = bertscore_metric.compute(predictions=predictions, references=references, lang="en")
    bertscore = {key: np.mean(bertscore[key]) for key in bertscore if key != "hashcode"}
    exact_match = exact_match_metric.compute(predictions=predictions, references=references)
    return {
        "bertscore": bertscore,
        "exact_match": exact_match,
        "success_execute": success_execute
        }

In [7]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

In [8]:
import jsonlines
from tqdm import tqdm

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

predictions = []
references = []

with jsonlines.open("/content/validation_set.jsonl", "r") as reader:
    for line in tqdm(list(reader)):
        inputs = tokenizer.apply_chat_template(
            line["conversations"][:2],
            tokenize = True,
            add_generation_prompt = True, # Must add for generation
            return_tensors = "pt",
        ).to("cuda")
        inputs_length = inputs.shape[1]

        outputs = model.generate(input_ids = inputs, max_new_tokens = 2056,
                   use_cache = True, temperature = 0.0, do_sample=False)


        results = tokenizer.batch_decode(outputs[:, inputs_length:])[0].removesuffix("<|eot_id|>")
        predictions.append(results)
        references.append(line["conversations"][-1]["content"])

100%|██████████| 178/178 [11:40<00:00,  3.94s/it]


In [81]:
results = compute_metrics(predictions=predictions, references=references)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [82]:
from pprint import pprint
pprint(results)

{'bertscore': {'f1': 0.9884676873014214,
               'precision': 0.9895306922746508,
               'recall': 0.9874512158083112},
 'exact_match': {'exact_match': 0.550561797752809},
 'success_execute': {'percent_different_meaning': 0.0,
                     'percent_invalid': 0.0056179775280898875,
                     'percent_same_meaning': 1.0,
                     'percent_valid': 0.9943820224719101}}
